In [29]:
import pandas as pd
import torch
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoModelForTokenClassification, AutoTokenizer, RagTokenizer, RagRetriever, RagSequenceForGeneration


ModuleNotFoundError: No module named 'torch'

In [24]:
df = pd.read_csv('FIR_DATASET.csv')

In [25]:
df.head()

,URL,Description,Offense,Punishment,Cognizable,Bailable,Court
0,https://lawrato.com/indian-kanoon/ipc/section-140,Description of IPC Section 140\nAccording to s...,Wearing the dress or carrying any token used b...,3 Months or Fine or Both,Cognizable,Bailable,Any Magistrate
1,https://lawrato.com/indian-kanoon/ipc/section-127,Description of IPC Section 127\nAccording to s...,Receiving property taken by war or depredation...,7 Years + Fine + forfeiture of property,Cognizable,Non-Bailable,Court of Session
2,https://lawrato.com/indian-kanoon/ipc/section-128,Description of IPC Section 128\nAccording to s...,Public servant voluntarily allowing prisoner o...,Imprisonment for Life or 10 Years + Fine,Cognizable,Non-Bailable,Court of Session
3,https://lawrato.com/indian-kanoon/ipc/section-129,Description of IPC Section 129\nAccording to s...,Public servant negligently suffering prisoner ...,Simple Imprisonment 3 Years + Fine,Cognizable,Bailable,Magistrate First Class
4,https://lawrato.com/indian-kanoon/ipc/section-130,Description of IPC Section 130\nAccording to s...,"Aiding escape of, rescuing or harbouring, such...",Imprisonment for Life or 10 Years + Fine,Cognizable,Non-Bailable,Court of Session


In [26]:
section_data = df['Description']
offense_data = df['Description']
offense_labels = df['Offense']

In [27]:
train_data, test_data, train_labels, test_labels = train_test_split(
    section_data, df['section'], test_size=0.2, random_state=42
)

NameError: name 'train_test_split' is not defined

In [28]:
bert_model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = TFBertForSequenceClassification.from_pretrained(bert_model_name)

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

C:\Users\Rebel Soul\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Rebel Soul\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

ImportError: 
TFBertForSequenceClassification requires the TensorFlow library but it was not found in your environment. Checkout the instructions on the
installation page: https://www.tensorflow.org/install and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [ ]:
train_encodings = tokenizer(list(train_data), truncation=True, padding=True, return_tensors='tf')
test_encodings = tokenizer(list(test_data), truncation=True, padding=True, return_tensors='tf')



In [ ]:
optimizer_bert = tf.keras.optimizers.Adam(learning_rate=5e-5)
bert_model.compile(optimizer=optimizer_bert, loss=bert_model.compute_loss, metrics=['accuracy'])
labels_bert = tf.keras.utils.to_categorical(train_labels, num_classes=len(df['section'].unique()))
history_bert = bert_model.fit(train_encodings, labels_bert, epochs=3, batch_size=8)


In [ ]:
bert_model.evaluate(test_encodings, tf.keras.utils.to_categorical(test_labels, num_classes=len(df['section'].unique())))


In [ ]:
llama2_model_path = 'path_to_llama2_model'  # Replace with the actual path to your LLama-2 model files
tokenizer_llama2 = AutoTokenizer.from_pretrained(llama2_model_path)
model_llama2 = AutoModelForTokenClassification.from_pretrained(llama2_model_path)

llama2_encodings = tokenizer_llama2(list(offense_data), truncation=True, padding=True, return_tensors='pt')


In [ ]:
optimizer_llama2 = torch.optim.AdamW(model_llama2.parameters(), lr=5e-5)
labels_llama2 = torch.tensor(list(offense_labels))  # Assuming you have labels for LLama-2
optimizer_llama2.zero_grad()
outputs_llama2 = model_llama2(**llama2_encodings, labels=labels_llama2)
loss_llama2 = outputs_llama2.loss
loss_llama2.backward()
optimizer_llama2.step()

In [ ]:
model_llama2.eval()
with torch.no_grad():
    logits_llama2 = model_llama2(**llama2_encodings).logits

In [ ]:
predicted_labels_bert = bert_model.predict(test_encodings)
predicted_labels_bert = tf.argmax(predicted_labels_bert['logits'], axis=1)


In [ ]:
predicted_labels_llama2 = torch.argmax(logits_llama2, dim=1)


In [30]:
rag_tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")
rag_retriever = RagRetriever.from_pretrained("facebook/rag-token-base")
rag_model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-base")



NameError: name 'RagTokenizer' is not defined

In [31]:
context = " ".join(offense_data.iloc[0:5]) 
input_ids = rag_tokenizer(context, return_tensors="pt").input_ids


NameError: name 'rag_tokenizer' is not defined

In [32]:
generated_output = rag_model.generate(input_ids)
generated_text = rag_tokenizer.decode(generated_output[0], skip_special_tokens=True)


NameError: name 'rag_model' is not defined

In [ ]:
print("Generated Text:", generated_text)